# EDA con Enfoque en CAC y LTV

Este cuaderno se centra en **preguntas de negocio** que generan valor al entender la relación entre:
- **CAC**: Costo de Adquisición de Cliente.
- **LTV**: Valor de Vida del Cliente.

Adicionalmente, explora aspectos clave como **suscripción**, **descuentos**, **categoría** del producto, **frecuencia de compras**, **método de pago**, etc. Con estas preguntas podrás tomar **decisiones estratégicas** sobre tu modelo de negocio, campañas de marketing y retención de clientes.

## Contenido:
1. **Importar librerías y cargar datos**
2. **Inspección inicial** (dimensiones, tipos de datos, nulos)
3. **Limpieza básica** (si es necesaria)
4. **Creación de columna** ratio `LTV/CAC` (rentabilidad por cliente)
5. **Preguntas de negocio** (groupby, filtros, etc.)
6. **Conclusiones**

In [ ]:
!wget !wget https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv

--2025-03-09 17:30:23--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2025-03-09 17:30:23--  https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv [following]
--2025-03-09 17:30:24--  https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HT

## 1. Importar librerías y cargar datos
Asegúrate de tener tu archivo CSV con las columnas mencionadas: `CAC`, `LTV`, `Purchase Amount (USD)`, etc. Ajusta la ruta según tu archivo real.

In [ ]:
import pandas as pd
import numpy as np


# Cargar el dataset
df = pd.read_csv("shopping_trends_LTV_CAC.csv")


# Mostramos las primeras filas para verificar
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,...,Next Day Air,Yes,Yes,49,PayPal,Weekly,NaN,1914,26.957746,26.957746
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574


## 2. Inspección inicial
1. **Dimensiones (filas x columnas)**
2. **Info general**: tipos de dato, nulos, etc.
3. **Valores nulos** por columna.

Si existen valores nulos en columnas importantes (por ejemplo, `CAC`, `LTV`, `Customer ID`), evalúa si los eliminarás o imputarás.

In [ ]:
df.shape

(3900, 23)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer ID               3900 non-null   int64  
 1   Age                       3900 non-null   int64  
 2   Gender                    3900 non-null   object 
 3   Item Purchased            3900 non-null   object 
 4   Category                  3900 non-null   object 
 5   Purchase Amount (USD)     3900 non-null   int64  
 6   Location                  3900 non-null   object 
 7   Size                      3900 non-null   object 
 8   Color                     3900 non-null   object 
 9   Season                    3900 non-null   object 
 10  Review Rating             3900 non-null   float64
 11  Subscription Status       3900 non-null   object 
 12  Payment Method            3900 non-null   object 
 13  Shipping Type             3900 non-null   object 
 14  Discount

In [ ]:
df.describe()

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
count,3900.000000,3900.000000,3900.000000,3900.000000,3900.000000,3301.000000,3900.000000,3900.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.749949,25.351538,66.292336,1108.944872,28.330611,23.301258
std,1125.977353,15.207589,23.685392,0.716223,14.447125,27.712380,527.645495,27.627277,22.705725
min,1.000000,18.000000,20.000000,2.500000,1.000000,10.000000,200.000000,2.070707,1.722689
25%,975.750000,31.000000,39.000000,3.100000,13.000000,44.000000,657.000000,11.665094,10.035880
50%,1950.500000,44.000000,60.000000,3.700000,25.000000,67.000000,1107.000000,19.784776,17.062261
75%,2925.250000,57.000000,81.000000,4.400000,38.000000,88.000000,1563.000000,33.307253,27.629467
max,3900.000000,70.000000,100.000000,5.000000,50.000000,120.000000,2410.000000,215.800000,215.800000


## 3. Limpieza básica
Por ejemplo, si detectas `CAC <= 0` o `LTV <= 0` y consideras que no tienen sentido en tu negocio, podrías filtrar esos registros. Ajusta según tu criterio.

In [ ]:
df = df[(df['CAC'] <= 0) & (df['LTV'] <= 0)]

## 4. Creación de columna: Ratio LTV/CAC
Es una métrica clave de la **rentabilidad** del cliente. Un valor > 1 indica que, en teoría, el cliente retorna más de lo que cuesta adquirirlo.

In [ ]:
df['Rentabilidad'] = df['LTV'] / df['CAC']
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,13.500000,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,15.757895,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,35.531250,35.531250
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,...,Next Day Air,Yes,Yes,49,PayPal,Weekly,NaN,1914,NaN,26.957746
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,27.659574,27.659574


## 5. Preguntas de Negocio
En vez de usar gráficas o correlaciones, nos centramos en **preguntas que generen valor** al análisis. Cada una se resuelve con **groupby, filtros o estadísticas** simples.

1. **¿Cuántos clientes únicos hay y cuál es la edad promedio?**
2. **¿Cuál es el CAC promedio y el LTV promedio en todo el dataset?**
3. **¿Cuál es la ratio LTV/CAC promedio?**
4. **¿Qué categoría tiene el mayor LTV en promedio?**
5. **¿En cuál categoría se justifica más el CAC?** (categoría con mayor ratio LTV/CAC)
6. **¿Los clientes suscritos tienen LTV mayor que los no suscritos?**
7. **¿Cuál es el método de pago más frecuente y su LTV promedio?**
8. **¿Quiénes usan descuentos con mayor frecuencia?** (Por ejemplo, ¿cierto género, cierto rango de edad?)
9. **¿Cuántos clientes aprovechan códigos promocionales (Promo Code Used)?** y su ratio LTV/CAC
10. **¿Cuál es la frecuencia de compra más común (Frequency of Purchases)?** y su LTV asociado.
11. **¿Cuál es la ubicación (Location) con mayor ratio LTV/CAC?**
12. **Entre los clientes con `Previous Purchases` > X, cuál es el gasto promedio y LTV promedio?**


### 5.1 ¿Cuántos clientes únicos hay y cuál es la edad promedio?

In [ ]:
clientesU = df['Customer ID'].nunique()
edadP = df['Age'].mean()

print("Número de clientes únicos: ", {clientesU})
print("Edad promedio de los clientes:", {edadP})

Número de clientes únicos:  {3900}
Edad promedio de los clientes: {44.06846153846154}


### 5.2 ¿Cuál es el CAC promedio y el LTV promedio en todo el dataset?

In [ ]:
CACP = df['CAC'].mean()
LTVP = df['LTV'].mean()

print("Costo de Adquisición Promedio (CAC):", {CACP})
print("Valor de Vida del Cliente Promedio (LTV):", {LTVP})

Costo de Adquisición Promedio (CAC): {66.29233565586186}
Valor de Vida del Cliente Promedio (LTV): {1108.9448717948717}


### 5.3 ¿Cuál es la ratio LTV/CAC promedio?

In [ ]:
Ratio = df['Rentabilidad'].mean()

print("Ratio LTV/CAC promedio:", {Ratio})

Ratio LTV/CAC promedio: {28.330610734927788}


### 5.4 ¿Qué categoría tiene el mayor LTV en promedio?

In [ ]:
categoriaMLRV = df.groupby('Category')['LTV'].mean().idxmax()

print("La categoría con el mayor LTV en promedio es:", {categoriaMLRV})

La categoría con el mayor LTV en promedio es: {'Accessories'}


### 5.5 ¿En cuál categoría se justifica más el CAC? (mayor ratio LTV/CAC)

In [ ]:
CategoriaCAC = df.groupby('Category')['Rentabilidad'].mean().idxmax()

print("La categoría que justifica más el CAC es:", {CategoriaCAC})

La categoría que justifica más el CAC es: {'Accessories'}


### 5.6 ¿Los clientes suscritos tienen LTV mayor que los no suscritos?

In [ ]:
CACP = df.groupby('Subscription Status')['LTV'].mean()

print("Los clientes suscritos tienen un LTV mayor que los no suscritos:", CACP)

Los clientes suscritos tienen un LTV mayor que los no suscritos: Subscription Status
No     1113.425009
Yes    1096.831909
Name: LTV, dtype: float64


### 5.7 ¿Cuál es el método de pago más frecuente y su LTV promedio?

In [ ]:
metodoPago = df.groupby('Payment Method')['LTV'].mean()
MP = df['Payment Method'].value_counts()
print("El método de pago más frecuente es:", MP)
print("El LTV promedio es:", metodoPago)

El método de pago más frecuente es: Payment Method
Credit Card      696
Venmo            653
Cash             648
PayPal           638
Debit Card       633
Bank Transfer    632
Name: count, dtype: int64
El LTV promedio es: Payment Method
Bank Transfer    1139.090190
Cash             1151.208333
Credit Card      1097.510057
Debit Card       1089.439179
PayPal           1087.108150
Venmo            1090.260337
Name: LTV, dtype: float64


### 5.8 ¿Quiénes usan descuentos con mayor frecuencia?
Podemos ver si hay algún grupo (por género, rango de edad, etc.) que destaca. A modo de ejemplo, segmentaremos por género:

In [ ]:
df.groupby('Gender')['Discount Applied'].value_counts()

Gender  Discount Applied
Female  No                  1248
Male    Yes                 1677
        No                   975
Name: count, dtype: int64

Podrías repetir el ejercicio segmentando por **Age** (definiendo rangos) o por **Location**.

In [ ]:
df.groupby('Location')['Discount Applied'].value_counts()

Location       Discount Applied
Alabama        No                  53
               Yes                 36
Alaska         No                  43
               Yes                 29
Arizona        No                  43
                                   ..
West Virginia  Yes                 40
Wisconsin      No                  39
               Yes                 36
Wyoming        No                  41
               Yes                 30
Name: count, Length: 100, dtype: int64

### 5.9 ¿Cuántos clientes aprovechan códigos promocionales y cuál es su ratio LTV/CAC?
El uso de promos puede modificar la rentabilidad.

In [ ]:

promoU = df[df['Promo Code Used'] == 'Yes'].shape[0]
ltv_cac_promo = df[df['Promo Code Used'] == 'Yes']['Rentabilidad'].mean()

print("Número de clientes que usan códigos promocionales: ",{promoU})
print("Ratio LTV/CAC promedio para clientes que usan códigos promocionales:" ,{ltv_cac_promo})


Número de clientes que usan códigos promocionales:  {1677}
Ratio LTV/CAC promedio para clientes que usan códigos promocionales: {22.70503584762745}


### 5.10 ¿Cuál es la frecuencia de compra más común (Frequency of Purchases) y su LTV?
Esto ayuda a ver si, por ejemplo, los clientes que compran semanalmente tienen un LTV más alto.

In [ ]:
PromedioLTV = df.groupby('Frequency of Purchases')['LTV'].mean()
Frecuencia = df.groupby('Frequency of Purchases')['LTV'].mean().idxmax()

print("La frecuencia de compra más común es:", Frecuencia)
print("El LTV promedio para esa frecuencia es:", PromedioLTV)

La frecuencia de compra más común es: Bi-Weekly
El LTV promedio para esa frecuencia es: Frequency of Purchases
Annually          1107.472028
Bi-Weekly         1182.279707
Every 3 Months    1072.965753
Fortnightly       1106.501845
Monthly           1087.001808
Quarterly         1098.229130
Weekly            1111.230056
Name: LTV, dtype: float64


### 5.11 ¿Cuál es la ubicación (Location) con mayor ratio LTV/CAC?

In [ ]:
df.groupby('Location')['Rentabilidad'].mean().idxmax()

'New York'

### 5.12 Entre los clientes con `Previous Purchases` > X, ¿cuál es el gasto promedio y el LTV promedio?
Define un umbral X según tu contexto. Por ejemplo, X = 20:

In [ ]:
X = 20
clientesX = df[df['Previous Purchases'] > X]
gastoPromX = clientesX['Purchase Amount (USD)'].mean()
ltvProm = clientesX['LTV'].mean()
print(f"Para clientes con más de {X} compras previas:")
print(f"Gasto promedio: {gastoPromX}")
print(f"LTV promedio: {ltvProm}")


Para clientes con más de 20 compras previas:
Gasto promedio: 59.697734074390766
LTV promedio: 1114.2992731936724


## 6. Conclusiones

En este Notebook, hemos abordado un **EDA orientado a negocio** en el que:
1. **Identificamos** la base total de clientes y su edad promedio.
2. **Calculamos** el CAC y LTV promedio, y la ratio LTV/CAC global.
3. **Segregamos** por categorías, suscripción, uso de descuentos, métodos de pago, etc.
4. **Resolvimos** preguntas de negocio que ayudan a:
   - **Identificar** cuáles categorías o ubicaciones ofrecen mejor rentabilidad (ratio LTV/CAC).
   - **Entender** si la suscripción eleva el LTV.
   - **Explorar** la efectividad de promos y descuentos.
   - **Diferenciar** frecuencias de compra y ver su impacto en LTV.
   - **Focalizar** inversiones de marketing en los segmentos más prometedores.

Este tipo de **análisis** proporciona las bases para **estrategias** de retención, adquisición y segmentación de clientes. ¡Espero que te sea útil para **generar valor** en tu negocio!